# Automated ML



In [1]:
import logging 
import os
import csv

from matplotlib import pyplot as plt 
import numpy as np
import pandas as pd

import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

print("SDK Version:",azureml.core.VERSION)

SDK Version: 1.18.0


## Dataset

### Overview

This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

It's a great dataset for evaluating regression models to predict the house price 



In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Avinash-AutoML'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FC4DFJP6G to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aml_compute_cluster = "Avinash-Test"

try:
    compute_target = ComputeTarget(workspace=ws,name = aml_compute_cluster)
    print('Found Exisitng cluster, use it ')
except ComputeTargetException:

    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',max_nodes = 4,)
    compute_target = ComputeTarget.create(ws,aml_compute_cluster)


compute_target.wait_for_completion(show_output=True)

Found Exisitng cluster, use it 
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
found = False

key = "house_sales"
description_text = "house sales prediction dataset"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(df,test_size = 0.2,random_state = 42)

## AutoML Configuration



In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping": True,
    "primary_metric" : 'normalized_mean_absolute_error',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task = 'regression',
    debug_log = 'automl_reg_errors.log',
    training_data = x_train,
    label_column_name = "price",
    **automl_settings
)

## Run Details



In [7]:
from azureml.widgets import RunDetails
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)




Running on local machine
Parent Run ID: AutoML_a6ec38e7-9680-4b79-866b-f62133f6cb79

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardi

In [10]:
RunDetails(remote_run).show()
remote_run.get_status()
remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_a6ec38e7-9680-4b79-866b-f62133f6cb79',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-12-06T19:03:11.306899Z',
 'endTimeUtc': '2020-12-06T19:23:38.146753Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"Avinash-AutoML","subscription_id":"e8f628b3-bb5b-4edf-947a-8637ca6ea7c2","resource_group":"aml-quickstarts-129522","workspace_name":"quick-starts-ws-129522","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"normalized_mean_absolute_error","task_type":"regression","data_script":null,"validation_size":0.0,

## Best Model





In [11]:
best_run,fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model.steps)


Run(Experiment: Avinash-AutoML,
Id: AutoML_a6ec38e7-9680-4b79-866b-f62133f6cb79_30,
Type: None,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                feature_sweeping_config=None, feature_sweeping_timeout=None,
                featurization_config=None, force_text_dnn=None,
                is_cross_validation=None, is_onnx_compatible=None, logger=None,
                observer=None, task=None, working_dir=None)), ('prefittedsoftvotingregressor', PreFittedSoftVotingRegressor(estimators=[('0',
                                          Pipeline(memory=None,
                                                   steps=[('maxabsscaler',
                                                           MaxAbsScaler(copy=True)),
                                                          ('lightgbmregressor',
                                                           LightGBMRegressor(boosting_type='gbdt',
                                   

In [12]:
#TODO: Save the best model
import joblib

joblib.dump(fitted_model,"housing_sales.pkl")


['housing_sales.pkl']

In [13]:
model = best_run.register_model(model_path = './outputs',model_name = 'house_sales1')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.



In [14]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")


inference_config = InferenceConfig(entry_script="scoring.py",environment=myenv)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    auth_enabled=True,
    memory_gb=1,
    tags = {'name':'housing sales'},
    description='housing sales model'
)

service = Model.deploy(workspace=ws,
                        name = 'house-sale',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=aci_config
)

service.wait_for_deployment(True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


Calling the Model Endpoint 

In [18]:
y_test = x_test.pop("price")
import json 
import requests

x_json = x_test[:5].to_json(orient = 'records')
parsed = json.loads(x_json)

#print(parsed)
test_samples = json.dumps({"data":parsed})

# send a random row from the test set to score

key1, Key2 = service.get_keys()

headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

result = requests.post(service.scoring_uri, test_samples, headers=headers)


print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[0:5])
#print("prediction:", resp.text)

print("predicted model output: ",result)


POST to url http://899f69d8-fe49-49c5-9fcf-5d8e615b784a.southcentralus.azurecontainer.io/score
label: 735      365000.00
2830     865000.00
4106    1038000.00
16218   1490000.00
19964    711000.00
Name: price, dtype: float64
predicted model output:  [401975.47936734813, 959191.3428419649, 1089646.7725140043, 1584312.9401446793, 687234.4692346373]


Printing the Logs and deleting the service

In [19]:
service.get_logs()


'2020-12-06T19:50:44.468557Z stdout F 2020-12-06T19:50:44,619602900+00:00 - gunicorn/run \n2020-12-06T19:50:44.468557Z stdout F 2020-12-06T19:50:44,618788500+00:00 - iot-server/run \n2020-12-06T19:50:44.4864279Z stdout F 2020-12-06T19:50:44,637069100+00:00 - rsyslog/run \n2020-12-06T19:50:44.5626152Z stdout F 2020-12-06T19:50:44,713621200+00:00 - nginx/run \n2020-12-06T19:50:44.5685821Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-06T19:50:44.5685821Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-06T19:50:44.5728203Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_99474c16f35a92149d1cc6dec149d8e9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-12-06T19:50:44.5728203Z stderr F /usr/sbin/

In [20]:
service.delete()